In [89]:
import pandas as pd
import numpy as np
import plotly.express as px

pd.set_option('display.max_columns', None)

In [90]:
raw_data = pd.read_csv("data/postings.csv")

In [91]:
raw_data.columns

Index(['job_id', 'company_name', 'title', 'description', 'max_salary',
       'pay_period', 'location', 'company_id', 'views', 'med_salary',
       'min_salary', 'formatted_work_type', 'applies', 'original_listed_time',
       'remote_allowed', 'job_posting_url', 'application_url',
       'application_type', 'expiry', 'closed_time',
       'formatted_experience_level', 'skills_desc', 'listed_time',
       'posting_domain', 'sponsored', 'work_type', 'currency',
       'compensation_type', 'normalized_salary', 'zip_code', 'fips'],
      dtype='object')

In [92]:
raw_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 123849 entries, 0 to 123848
Data columns (total 31 columns):
 #   Column                      Non-Null Count   Dtype  
---  ------                      --------------   -----  
 0   job_id                      123849 non-null  int64  
 1   company_name                122130 non-null  object 
 2   title                       123849 non-null  object 
 3   description                 123842 non-null  object 
 4   max_salary                  29793 non-null   float64
 5   pay_period                  36073 non-null   object 
 6   location                    123849 non-null  object 
 7   company_id                  122132 non-null  float64
 8   views                       122160 non-null  float64
 9   med_salary                  6280 non-null    float64
 10  min_salary                  29793 non-null   float64
 11  formatted_work_type         123849 non-null  object 
 12  applies                     23320 non-null   float64
 13  original_liste

In [93]:
raw_data["pay_period"].value_counts()

pay_period
YEARLY      20628
HOURLY      14741
MONTHLY       518
WEEKLY        177
BIWEEKLY        9
Name: count, dtype: int64

In [94]:
# CREATE A NEW YEARLY SALARY COLUMN FROM EXISTING SALARY COLUMNS TO STANDARDIZE THIS METRIC ACROSS THE DATASET 

salary_columns: list[str] = ["max_salary","med_salary","min_salary"]
salary_period_type_column: str = "pay_period"

def convert_to_yearly_salary(df: pd.DataFrame) -> pd.DataFrame:
    df = df.copy()

    df.loc[df[salary_period_type_column]=="HOURLY", salary_columns] = df.loc[df[salary_period_type_column]=="HOURLY", salary_columns]*2080
    df.loc[df[salary_period_type_column]=="WEEKLY", salary_columns] = df.loc[df[salary_period_type_column]=="WEEKLY", salary_columns]*52
    df.loc[df[salary_period_type_column]=="BIWEEKLY", salary_columns] = df.loc[df[salary_period_type_column]=="BIWEEKLY", salary_columns]*26
    df.loc[df[salary_period_type_column]=="MONTHLY", salary_columns] = df.loc[df[salary_period_type_column]=="MONTHLY", salary_columns]*12

    df["standardized_salary"] = df["med_salary"]
    
    df["avg_min_max"] = (df["max_salary"]+df["min_salary"])/2
    df.loc[df["standardized_salary"].isna()==True, "standardized_salary"] = df.loc[df["standardized_salary"].isna()==True,"avg_min_max"]

    return df

In [139]:
jobs_data = convert_to_yearly_salary(raw_data)

In [140]:
jobs_data = jobs_data[jobs_data["standardized_salary"]>=0]

In [141]:
COLUMNS_TO_KEEP = ['company_name'
                   ,'title'
                   ,'description'
                   ,'location'
                   ,'remote_allowed'
                   ,'work_type'
                   ,'currency'
                   ,'standardized_salary']

jobs_data = jobs_data[COLUMNS_TO_KEEP]

In [142]:
# WE WILL FOCUS ON USD CURRENCY
# value_counts: 
# USD    36058
# EUR        6
# CAD        3
# BBD        2
# AUD        2
# GBP        2

jobs_data = jobs_data[jobs_data["currency"]=="USD"]

In [143]:
# WE WILL FOCUS ON FULL_TIME, CONTRACT, PART_TIME WORK TYPES
# value_counts: 
# FULL_TIME     29119
# CONTRACT       3848
# PART_TIME      2304
# TEMPORARY       394
# INTERNSHIP      247
# OTHER           138
# VOLUNTEER         8

jobs_data = jobs_data[jobs_data["work_type"].isin(["FULL_TIME","CONTRACT","PART_TIME"])]

In [144]:
jobs_data["location"].str.split(',',expand=True)[1].nunique()

95

In [145]:
jobs_data["state"]=jobs_data["location"].str.split(',',expand=True)[1].str.strip()

In [150]:
location_renaming = {
    'United States': 'US'
    ,'Ohio Metropolitan Area': 'OH'
    ,'Texas Metropolitan Area': 'TX'
    ,'California': 'CA'
    ,'South Carolina Metropolitan Area': 'SC'
    ,'Oregon Metropolitan Area': 'OR'
    ,'Alabama Area': 'AL'
    ,'Kansas Metropolitan Area': 'KS'
    ,'Massachusetts Metropolitan Area': 'MI'
    ,'Colorado': 'CO'
    ,'Nebraska Metropolitan Area': 'NE'
    ,'New York': 'NY'
    ,'New York Metropolitan Area': 'NY'
    ,'MI Area': 'MI'
    ,'Illinois': 'IL'
    ,'Texas': 'TX'
    ,'Louisiana Metropolitan Area': 'LA'
    ,'South Carolina Area': 'SC'
    ,'Hawaii': 'HI'
    ,'Ohio': 'OH'
    ,'Virginia Metropolitan Area': 'VA'
    ,'Florida': 'FL'
    ,'Delaware': 'DE'
    ,'Arizona': 'AZ'
    ,'North Carolina': 'NC'
    ,'New Jersey': 'NJ'
    ,'North Carolina Metropolitan Area': 'NC'
    ,'Illinois Metropolitan Area': 'IL'
    ,'Alaska': 'AK'
    ,'Nebraska': 'NE'
    ,'Georgia': 'GA'
    ,'Washington': 'WA'
    ,'Massachusetts': 'MA'
    ,'AR Area': 'AR'
    ,'Wisconsin Metropolitan Area': 'WI'
    ,'Maine Metropolitan Area': 'ME'
    ,'Oregon': 'OR'
    ,'Indiana Metropolitan Area': 'IN'
    ,'Oklahoma': 'OK'
    ,'Michigan': 'MI'
    ,'Utah': 'UT'
    ,'Georgia Area': 'GA'
    ,'New Mexico': 'NM'
    ,'Virginia': 'VA'
}

In [151]:
jobs_data["state"] = jobs_data["state"].replace(location_renaming)

In [153]:
jobs_data["state"].nunique()

52